In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.externals import joblib

In [2]:
train = pd.read_csv('../data/preprocessed/train.csv')
test = pd.read_csv('../data/preprocessed/test.csv')

In [3]:
features = ['count_sent',
            'count_word',
            'count_letters',
            'count_punctuations',
            'mean_word_len',
            'word_unique_percent',
            'uppercase_percent',
            'stopwords_perent']

target = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [4]:
def calc_avg_auc(Y,predictions):
    nb_classes = Y.shape[1]
    cols = Y.columns
    score = 0
    for cl in range(nb_classes):
        score += roc_auc_score(Y[cols[cl]],predictions[:,cl])
    score = score/nb_classes
    return score

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(train[features],train[target], random_state=997)
X = test[features]

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [6]:
logit = [None]*len(target)
for cl in range(len(target)):
    logit[cl] = LogisticRegression(multi_class = 'ovr', class_weight = 'balanced')
    logit[cl].fit(X_train,Y_train[list(Y_train.columns)[cl]])

In [7]:
predictions = np.zeros((X_test.shape[0],len(target)))
for cl in range(len(target)):
    predictions[:,cl] = logit[cl].predict(X_test)
score = calc_avg_auc(Y_test,predictions)
print("Mean AUC for a set of single-target logit models is " + str(score))

Mean AUC for a set of single-target logit models is 0.620174187236


In [8]:
submission_logit = np.zeros((X.shape[0],len(target)))
for cl in range(Y_train.shape[1]):
    submission_logit[:,cl] = logit[cl].predict(X)
submission_logit = pd.DataFrame(submission_logit, columns = target)
submission_logit['id'] = test['id']
submission_logit=submission_logit[['id']+target]
submission_logit.to_csv('../submissions/logit.csv', index = False)

# Tree-Based Models

## Random Forest (RF)

In [7]:
from sklearn.tree import *
from sklearn.ensemble import *

### One RF per class

In [33]:
rf = [None]*len(target)
for cl in range(Y_train.shape[1]):
    rf[cl] = RandomForestClassifier(n_estimators=130, max_features=None,random_state=997,min_samples_leaf=100)
    rf[cl].fit(X_train,Y_train[list(Y_train.columns)[cl]])

In [34]:
predictions = np.zeros((X_test.shape[0],len(target)))
for cl in range(len(target)):
    predictions[:,cl] = rf[cl].predict_proba(X_test)[:,1]
score = calc_avg_auc(Y_test,predictions)
print("Mean AUC for a set of single-target RF models is " + str(score))

Mean AUC for a set of single-target RF models is 0.770897555977


In [35]:
submission_rf = np.zeros((X.shape[0],len(target)))
for cl in range(Y_train.shape[1]):
    submission_rf[:,cl] = rf[cl].predict_proba(X)[:,1]
    joblib.dump(rf[cl],'../models/rf'+target[cl]+'.pkl')
submission_rf = pd.DataFrame(submission_rf, columns = target)
submission_rf['id'] = test['id']
submission_rf=submission_rf[['id']+target]
submission_rf.to_csv('../submissions/rf.csv', index = False)

### Single multi-output RF

In [43]:
rf_joint = RandomForestClassifier(n_estimators=130,criterion='gini',random_state=997,class_weight='balanced',min_samples_leaf=50)
rf_joint.fit(X_train,Y_train)
predictions = rf_joint.predict_proba(X_test)
predictions_matrix = np.zeros((X_test.shape[0],len(target)))
for targ in range(len(target)):
    predictions_matrix[:,targ] = predictions[targ][:,1]
score = calc_avg_auc(Y_test,predictions_matrix)
print("Mean AUC for a single multi-target RF model is " + str(score))

Mean AUC for a single multi-target RF model is 0.737785683318


In [44]:
for cl in range(Y_test.shape[1]):
    cols = Y_test.columns
    auc = roc_auc_score(Y_test[cols[cl]],predictions_matrix[:,cl])
    print("Class "+cols[cl]+", AUC = "+str(auc))

Class toxic, AUC = 0.702739793752
Class severe_toxic, AUC = 0.807153090431
Class obscene, AUC = 0.735892097635
Class threat, AUC = 0.730944133417
Class insult, AUC = 0.728980487772
Class identity_hate, AUC = 0.721004496899


In [29]:
probabilities = rf_joint.predict_proba(X)
submission_rf_joint = np.zeros((X.shape[0],len(target)))
for targ in range(len(target)):
    submission_rf_joint[:,targ] = probabilities[targ][:,1]
submission_rf_joint = pd.DataFrame(submission_rf_joint, columns = target)
submission_rf_joint['id'] = test['id']
submission_rf_joint=submission_rf_joint[['id']+target]
submission_rf_joint.to_csv('../submissions/evgeniya/rf_joint_50samples.csv', index = False)
joblib.dump(rf_joint,'../models/rf_joint_50samples.pkl')

['../models/rf_joint_50samples_entropy.pkl']

## AdaBoost

In [36]:
from sklearn.ensemble import AdaBoostRegressor

In [40]:
ada = [None]*len(target)
base_estimator = DecisionTreeClassifier(min_samples_leaf=100)
for cl in range(Y_train.shape[1]):
    ada[cl] = AdaBoostClassifier(base_estimator=base_estimator)
    ada[cl].fit(X_train,Y_train[list(Y_train.columns)[cl]])

In [41]:
predictions = np.zeros((X_test.shape[0],len(target)))
for cl in range(len(target)):
    predictions[:,cl] = ada[cl].predict_proba(X_test)[:,1]
    joblib.dump(rf[cl],'../models/ada'+target[cl]+'.pkl')
score = calc_avg_auc(Y_test,predictions)
print("Mean AUC for a set of AdaBoost models is " + str(score))

Mean AUC for a set of AdaBoost models is 0.716101547481


In [42]:
submission_ada = np.zeros((X.shape[0],len(target)))
for cl in range(Y_train.shape[1]):
    submission_ada[:,cl] = ada[cl].predict_proba(X){:,1}
submission_ada = pd.DataFrame(submission_ada, columns = target)
submission_ada['id'] = test['id']
submission_ada=submission_ada[['id']+target]
submission_ada.to_csv('../submissions/ada.csv', index = False)

SyntaxError: invalid syntax (<ipython-input-42-897d76b2958f>, line 3)

## XGBoost

In [47]:
from xgboost import XGBClassifier

ImportError: No module named xgboost

In [ ]:
xgboost = [None]*len(target)
for cl in range(Y_train.shape[1]):
    xgboost[cl] =XGBBoostClassifier()
    xgboost[cl].fit(X_train,Y_train[list(Y_train.columns)[cl]])